In [1]:
%load_ext autoreload
%autoreload 2

In [15]:
import itertools
import json

from sklearn.metrics import cohen_kappa_score
import torch

import emphases

with open(emphases.CACHE_DIR / 'crowdsource' / 'annotations.json') as file:
    contents = json.load(file)
    annotators, annotations = zip(*contents.items())

data = torch.zeros((len(annotations), len(annotations)))
counts = torch.zeros((len(annotations), len(annotations)), dtype=torch.long)

# Calculate cohen_kappa_score for every combination of raters
# Combinations are only calculated j -> k, but not k -> j, which are equal
# So not all places in the matrix are filled.
for j, k in list(itertools.combinations(range(len(annotators)), r=2)):
    a, b = annotations[j]['annotations'], annotations[k]['annotations']
    matches = [(x, y) for x in a for y in b if x['stem'] == y['stem']]
    if not matches:
        data[j, k] = data[k, j] = float('nan')
    else:
        flat_a = torch.cat([torch.tensor(match[0]['score']) for match in matches])
        flat_b = torch.cat([torch.tensor(match[1]['score']) for match in matches])
        data[j, k] = data[k, j] = cohen_kappa_score(flat_a, flat_b)
        counts[j, k] = counts[k, j] = len(matches)

for i in range(len(data)):
    data[i, i] = 1.

/home/mrm5248/conda/envs/env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:697: RuntimeWarning: invalid value encountered in double_scalars
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
/home/mrm5248/conda/envs/env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:697: RuntimeWarning: invalid value encountered in double_scalars
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
/home/mrm5248/conda/envs/env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:697: RuntimeWarning: invalid value encountered in double_scalars
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
/home/mrm5248/conda/envs/env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:697: RuntimeWarning: invalid value encountered in double_scalars
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
/home/mrm5248/conda/envs/env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:697: RuntimeWarning: invalid value encounter

In [47]:
total, count = 0., 0
for j, k in list(itertools.combinations(range(len(annotators)), r=2)):
    datum = data[j, k]
    if not torch.isnan(datum):
        count += 1
        total += datum
total / count

tensor(0.2259)

In [36]:
irt_input = []
with open(emphases.CACHE_DIR / 'crowdsource' / 'annotations.json') as file:
    for annotator, contents in json.load(file).items():
        responses = {}
        for annotation in contents['annotations']:
            stem = annotation['stem']
            for i, score in enumerate(annotation['score']):
                responses[f'{stem}-{i}'] = int(score)
        irt_input.append({
            'subject_id': annotator,
            'responses': responses})

with open('irt/irt_input.jsonlines', 'w') as file:
    for item in irt_input:
        json.dump(item, file)
        file.write('\n')

In [38]:
!py-irt train 1pl irt/irt_input.jsonlines irt/


[14:42:26] config: model_type='1pl' epochs=2000 priors=None           ]8;id=1694443346.113146-116149;file:///home/mrm5248/conda/envs/env/lib/python3.9/site-packages/py_irt/cli.py\cli.py]8;;\:109
           initializers=[] dims=None lr=0.1 lr_decay=0.9999                     
           dropout=0.5 hidden=100 vocab_size=None log_every=100                 
           seed=None deterministic=False                                        
           data_path: irt/irt_input.jsonlines                         ]8;id=1694443346.1143017-10272;file:///home/mrm5248/conda/envs/env/lib/python3.9/site-packages/py_irt/cli.py\cli.py]8;;\:111
           output directory: irt/                                     ]8;id=1694443346.1151235-270636;file:///home/mrm5248/conda/envs/env/lib/python3.9/site-packages/py_irt/cli.py\cli.py]8;;\:112
[14:42:26] amortized: False                                       ]8;id=1694443346.2957995-930848;file:///home/mrm5248/conda/envs/env/lib/python3.9/site-packa

In [30]:
import json

correct, count = 0, 0
with open('irt/model_predictions.jsonlines') as file:
    for row in file.readlines():
        content = json.loads(row)
        correct += int(round(content['prediction'])) == int(content['response'])
        count += 1

In [31]:
correct / count

0.7769449470147325

In [41]:
count

19345